In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
#Generate Dummy Data
np.random.seed(0)
N = 50
data = pd.DataFrame({
    'area': np.random.randint(800, 3000, N),
    'rooms': np.random.randint(1, 5, N),
    'age': np.random.randint(1, 20, N),
})

In [ ]:
data['price'] = (
    data['area'] * 200 +
    data['rooms'] * 15000 -
    data['age'] * 800 +
    np.random.normal(0, 20000, N)
)
#Discretize Target (Regression Classification)
bins = [0, 500_000, 900_000, np.inf]
labels = [0, 1, 2]  # 0=low, 1=medium, 2=high
data['price_class'] = pd.cut(data['price'], bins=bins, labels=labels).astype(int)

In [ ]:
X = data[['area', 'rooms', 'age']]
y = data['price_class']
n_classes = len(y.unique())

# Train/Test split (preserve indices)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to NumPy only for training math
X_train_np = X_train.values
y_train_np = y_train.values
X_test_np = X_test.values
y_test_np = y_test.values

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
#Training (Gradient Descent for Each Class - One-vs-Rest)
def train_one_vs_rest(X, y, n_classes, lr=1e-7, n_iter=10000):

    n_samples, n_features = X.shape
    weights = []
    for k in range(n_classes):
        # Binary labels for class k
        yk = (y == k).astype(np.float64)
        w = np.zeros(n_features)
        b = 0.0
        for _ in range(n_iter):
            logits = np.dot(X, w) + b
            y_pred = sigmoid(logits)
            # Gradients
            dw = np.dot(X.T, (y_pred - yk)) / n_samples
            db = np.sum(y_pred - yk) / n_samples
            # Update
            w -= lr * dw
            b -= lr * db
        weights.append({'w': w, 'b': b})
    return weights

def predict(X, weights):
    """Predict class labels for given features"""
    n_samples = X.shape[0]
    n_classes = len(weights)
    probs = np.zeros((n_samples, n_classes))
    for k in range(n_classes):
        w = weights[k]['w']
        b = weights[k]['b']
        logits = np.dot(X, w) + b
        probs[:, k] = sigmoid(logits)
    return np.argmax(probs, axis=1)

In [ ]:
#Train from scratch
weights = train_one_vs_rest(X_train, y_train, n_classes)


In [ ]:
y_pred = predict(X_test, weights)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.4

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.40      1.00      0.57         4

    accuracy                           0.40        10
   macro avg       0.20      0.50      0.29        10
weighted avg       0.16      0.40      0.23        10


Confusion Matrix:
 [[0 6]
 [0 4]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:


# 6. Sample Predictions Table (with correct true_price)
results = X_test.copy()
results['true_price'] = data.loc[X_test.index, 'price']
results['true_class'] = y_test
results['pred_class'] = y_pred
print(results.head())


#Class distribution

print("\nClass distribution:")
print(data['price_class'].value_counts())

    area  rooms  age     true_price  true_class  pred_class
16  1555      3    9  379284.536255           0           1
36  1556      1    3  284985.686480           0           1
22  1597      1    7  309518.732823           0           1
26  1344      4   14  305883.696352           0           1
15  1776      4   12  370738.835382           0           1

Class distribution:
price_class
0    28
1    22
Name: count, dtype: int64
